### Итоговая работа
### 12448/1 ИНЖДАН МГТУ им. Н.Э. Баумана
### Инженер данных (Data engineer)
### Андреев Андрей Анатольевич

## КЛАССИФИКАЦИЯ ГОСКОНТРАКТОВ ПО ОБЪЕКТАМ ЗАКУПКИ

#### Задача: необходимо на основе данных с ftp.zakupki.gov.ru научиться определять группу, к которой относится контракт с кодом ОКПД-2 41, 42, 43, 71.1.

Группы могут быть следующими:

1.	Строительно-монтажные работы (СМР)
2.	Проектно-изыскательские работы (ПИР)
3.	Строительный надзор
4.	Подключение коммуникаций
5.	Прочее.

По ОКПД-2 контракты в общем случае должны разделяться так:
- Строительно-монтажные работы (СМР) - 41, 42, 43(кроме нижеперечисленных)
- Проектно-изыскательские работы (ПИР) - 41.1, 71.1
- Подключение коммуникаций - 43.22
- Строительный надзор – четкой группы нет.


Проблема: Далеко не всегда контракты указываются с нужным кодом, поэтому есть проблема как такие контракты "отловить" и определить в нужную группу.

Поэтому задача предполагает классификацию контрактов на основе объекта закупки, который сформулирован естественным языком. Также предполагаем, что могут иметь значение цена контракта и его длительность.
На основе этого на входе данные о контрактах. На выходе необходимо получить группу для каждого контракта.

Иногда контракт может относиться одновременно в несколько групп.

В приложении ниже пример нескольких контрактов, у которых неверно проставлен ОКПД-2.

# Шаг 3 - NLP процесс

In [2]:
! pip install scikit-learn
! pip install nltk
! pip install pymorphy2
! pip install requests
! pip install gensim
! pip install pymystem3
! pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import numpy as np

In [4]:
columns = ['id', 'contract_number', 'object_name', 'object_code', 'cost', 'contract_execution_days']
dtypes = {'id': lambda x: pd.to_numeric(x, errors="coerce"), 
          'contract_number': str, 
          'object_name': str,
          'object_code': str, 
          'cost':lambda x: pd.to_numeric(x, errors="coerce"), 
          'contract_execution_days': lambda x: pd.to_numeric(x, errors="coerce")}

In [5]:
df = pd.read_csv('datasets/dataset_clean.csv', sep=';', low_memory=False, on_bad_lines='skip')
df.shape

(14197786, 6)

In [6]:
df.describe()

,id,contract_number,object_code,cost,contract_execution_days
count,1.419779e+07,1.419779e+07,1.419724e+07,1.419779e+07,1.419779e+07
mean,1.591788e+07,2.751354e+18,3.662128e+01,2.201673e+06,2.204985e+02
std,1.029077e+07,7.006991e+17,2.251888e+01,9.988996e+07,1.534003e+02
min,1.373130e+05,1.423000e+17,1.100000e+00,1.000000e-02,1.000000e+00
25%,6.804842e+06,2.360900e+18,2.120000e+01,3.825000e+04,9.600000e+01
50%,1.489233e+07,2.772393e+18,3.310000e+01,1.497160e+05,2.000000e+02
75%,2.398329e+07,3.343301e+18,4.390000e+01,5.626637e+05,3.430000e+02
max,3.509658e+07,4.920456e+18,9.900000e+01,1.425579e+11,2.000000e+03


In [7]:
df['object_name']  = df['object_name'].astype('string')
df['object_code']  = df['object_code'].astype('string')
df.dtypes

id                                  int64
contract_number                     int64
object_name                string[python]
object_code                string[python]
cost                              float64
contract_execution_days             int64
dtype: object

In [8]:
df.head()

,id,contract_number,object_name,object_code,cost,contract_execution_days
0,137313,2590229330821001063,"Средства лекарственные и материалы, применяемы...",21.2,342150.00,21
1,137314,2590229330821001064,"Средства лекарственные и материалы, применяемы...",21.2,342150.00,18
2,137315,2590229330821001065,"Средства лекарственные и материалы, применяемы...",21.2,342150.00,18
3,137320,2590229330821001067,"Средства лекарственные и материалы, применяемы...",21.2,26457423.22,389
4,137324,2590229330821001070,"Средства лекарственные и материалы, применяемы...",21.2,5756951.20,389


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14197786 entries, 0 to 14197785
Data columns (total 6 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   id                       int64  
 1   contract_number          int64  
 2   object_name              string 
 3   object_code              string 
 4   cost                     float64
 5   contract_execution_days  int64  
dtypes: float64(1), int64(3), string(2)
memory usage: 649.9 MB


In [10]:
grouped_data = df.groupby('object_code')['object_code'].unique()
grouped_data

object_code
1.1      [1.1]
1.2      [1.2]
1.3      [1.3]
1.4      [1.4]
1.6      [1.6]
         ...  
96.0    [96.0]
97.0    [97.0]
98.1    [98.1]
98.2    [98.2]
99.0    [99.0]
Name: object_code, Length: 272, dtype: object

По ОКПД-2 контракты в общем случае должны разделяться так:

0. Прочие
1. Строительно-монтажные работы (СМР) - 41, 42, 43(кроме нижеперечисленных)
2. Проектно-изыскательские работы (ПИР) - 41.1, 71.1
3. Подключение коммуникаций - 43.22
4. Строительный надзор – четкой группы нет. 

Строительный надзор - (43.9, 42.9, 42.2) - получил на основе ручного анализа данных. Лучше всего воспользоваться специалистом в этой предметной области.


In [11]:
query = '''object_code.str.contains("41.") or object_code.str.contains("42.") or object_code.str.contains("43.") or object_code == "71.1" '''
df = df.query(query, engine='python')

In [12]:
df.describe()

,id,contract_number,cost,contract_execution_days
count,1.077891e+06,1.077891e+06,1.077891e+06,1.077891e+06
mean,1.567699e+07,3.027632e+18,1.252777e+07,1.657274e+02
std,1.027531e+07,6.623508e+17,3.005228e+08,1.649801e+02
min,1.374390e+05,3.403001e+17,1.000000e-02,1.000000e+00
25%,7.038131e+06,2.770541e+18,2.309079e+05,6.300000e+01
50%,1.441286e+07,3.240700e+18,7.909571e+05,1.210000e+02
75%,2.204623e+07,3.504506e+18,2.522650e+06,2.140000e+02
max,3.509656e+07,4.920456e+18,1.425579e+11,2.000000e+03


Очистим описания услуг от латинских символов и чисел для дальнейшего анализа данных.

In [13]:
def clear_text(text):
    lst = [char for char in text if 1039 < ord(text[0])]
    return ''.join(lst)

In [14]:
df['object_name'] = df['object_name'].apply(lambda x: clear_text(x))
df = df[df["object_name"] != ""]

In [15]:
df.describe()

,id,contract_number,cost,contract_execution_days
count,9.001530e+05,9.001530e+05,9.001530e+05,900153.000000
mean,1.215062e+07,3.044760e+18,7.050218e+06,153.150148
std,7.133365e+06,6.505005e+17,1.685426e+08,146.767441
min,1.374390e+05,3.403001e+17,1.000000e-02,1.000000
25%,5.810018e+06,2.771570e+18,2.092741e+05,59.000000
50%,1.190011e+07,3.244301e+18,6.920000e+05,112.000000
75%,1.811065e+07,3.504601e+18,2.161987e+06,203.000000
max,2.719647e+07,4.920456e+18,1.387754e+11,2000.000000


In [16]:
df.to_csv('datasets/dataset_filter.csv', sep=';', index=False)

# NLP анализ

### Разбираем строки на токены, лематизируем и очищаем от шума (от стослов и знаков пунктуации)

In [17]:
#Получим стоп слова русского языка
import nltk
#nltk.download()
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\head\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk import download
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import defaultdict, Counter
from nltk.corpus import wordnet as wn  # Проверить необходимость
from nltk.corpus import stopwords

#print(stopwords.words('russian'))

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

stemmer = nltk.stem.snowball.RussianStemmer()
word_lemmatized = WordNetLemmatizer()


class NLP:
  
  def __init__(self, __df):
     self.df = __df

  def tokening(self):
     self.df['data'] =  [word_tokenize(entry.lower()) for entry in self.df['object_name']]

  def lammatize(self, word, tag):
     return word_lemmatized.lemmatize(word, tag_map[tag])

  def steming(self, word_final):
     return stemmer.stem(word_final)

  def fit(self): 
    self.tokening()
    for index, entry in zip(self.df["data"].index, self.df['data']):
        final_words = []
        for word, tag in pos_tag(entry):
            if word not in stopwords.words("russian") and word.isalpha():
                word_final = self.lammatize(word, tag[0])
                word_final = self.steming(word_final)
                final_words.append(word_final)
        self.df.loc[index, "data_final"] = str(final_words)
    return self.df

nlp = NLP(df)
vectors = nlp.fit()
vectors


C:\Users\head\AppData\Local\Temp\ipykernel_23728\3407045905.py:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['работ', 'строительн', 'специализирова']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  self.df.loc[index, "data_final"] = str(final_words)


,id,contract_number,object_name,object_code,cost,contract_execution_days,data,data_final
68,137439,2590407717221000153,Работы строительные специализированные,43.9,5.392656e+05,29,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']"
122,137528,2590421285821000134,Работы строительные специализированные,43.9,2.299470e+06,122,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']"
123,137529,2590421285821000135,Работы строительные специализированные,43.9,1.898112e+06,122,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']"
304,137827,2590500335021000676,Работы строительные специализированные,43.3,3.419140e+05,199,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']"
452,138113,2590615606021000060,Услуги в области архитектуры и инженерно-техни...,71.1,1.714352e+06,383,"[услуги, в, области, архитектуры, и, инженерно...","['услуг', 'област', 'архитектур', 'проектирова..."
...,...,...,...,...,...,...,...,...
12256223,27196374,3860104675919000035,Работы строительные специализированные,43.3,1.873113e+06,211,"[работы, строительные, специализированные]","['работ', 'строительн', 'специализирова']"
12256284,27196459,3860200041119000073,Сооружения и строительные работы в области гра...,42.1,1.430199e+08,374,"[сооружения, и, строительные, работы, в, облас...","['сооружен', 'строительн', 'работ', 'област', ..."
12256285,27196462,3860200041119000074,Сооружения и строительные работы в области гра...,42.1,1.161912e+08,374,"[сооружения, и, строительные, работы, в, облас...","['сооружен', 'строительн', 'работ', 'област', ..."
12256286,27196466,3860200041119000075,Сооружения и строительные работы в области гра...,42.1,7.774969e+07,738,"[сооружения, и, строительные, работы, в, облас...","['сооружен', 'строительн', 'работ', 'област', ..."


In [19]:
vectors.to_csv('datasets/dataset_nlp.csv', sep=';', index=False)